# Chatbot que é o Homer Simpson usando tf Seq2Seq
![alt text](https://drive.google.com/uc?id=1oU6A7qPq7gn4T342pxqQep0i1RszRSka)

## 1. Motivação
Quando ouvimos que o enunciado do projeto final era criar "qualquer" chatbot, utilizando "qualquer" tecnologia, logo tivemos a brilhante ideia de criar um modelo de redes neurais que aprende a conversar, claro, se conseguíssemos um conjunto de dados grande o bastante...


Um de nós então se lembrou do conjunto de dados de falas dos episódios de Os Simpsons desde 1989 até 2015 - e assim começou a nossa jornada para criar uma IA cujo único propósito de existência é ser o Homer Simpson.

![fig 1.1: expectativa](https://drive.google.com/uc?id=1QK2evq6RIFyav6zEHhTKXto1zBLZz_T2)

## 2. Introdução
Nós resolvemos fazer o ChatBot utilizando o modelo de redes neurais _Seq2Seq_, e mais tarde, após resultados não muito promissores ~~(spoiler)~~, utilizando o modelo _Doc2Vec_. Para entender por que nenhum dos dois modelos deu muito certo ~~(spoiler 2)~~, vamos primeiro dar uma olhada no conjunto de dados, e mais tarde nos dois modelos. 

Depois, fazemos experimentações com outras pessoas, e por fim, concluimos fazendo uma ligação do que foi feito aqui com o que foi visto na cadeira de Linguagens Formais e Autômatos.

Porém, antes que possamos prosseguir, caso o leitor deseje rodar os códigos, é preciso que copie a [pasta do drive do projeto](https://drive.google.com/open?id=1cs3x5OpAML9lGeDbHjby7GSifKu0X5P_), e o cole dentro da pasta principal do seu Google Drive (My Drive), pois todas as células de código tomam este diretório e seus arquivos como referência.

### 2.1. Conjunto de Dados
O conjunto pode ser encontrado [aqui](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data#simpsons_script_lines.csv), e contém falas de mais ou menos 600 episódios, totalizando mais de 100 mil falas.

![alt text](https://drive.google.com/uc?id=1VcaPZlBaODBt1qB7IH-dQ8vpTaLlHJQz)

Como pode-se ver, cada linha corresponde a uma fala, e possui 13 colunas (algumas escondidas na imagem), com diversas informações que podem ser potencialmente úteis, como o local, quem fala, e claro, a própria fala.

Você pode estar pensando "~~$@#&@#~~ 100k falas é muita fala!", mas eu vou explicar porque não é o caso:
- No [tutorial de tradução neural do TensorFlow](https://github.com/tensorflow/nmt), o conjunto de dados chamado de "Small Scale" possui 133k pares de falas
- No caso do modelo _Seq2Seq_, é preciso separar uma parte do conjunto de dados para ser o conjunto de validação (mais sobre isso a seguir), e não é usado para treinar a IA.
- Nem todas as falas são falas utilizáveis para nossos propósitos:
  - Algumas nem são de diálogo
  - Como a IA vai imitar o Homer, só podemos usar as falas dele, e as falas de outros personagens para ele. (no caso do modelo Doc2Vec podemos usar todas falas para treinar, mais sobre isso adiante)
  - Como os dois modelos recebem frases como entrada, devemos concatenar falas consequintes (do homer e para o homer) em uma só.

Após todos estes filtros, nos resta apenas 15k pares de falas, o que pode ser considerado insuficiente para treinar uma rede neural do zero para simular um personagem como o Homer.

### 2.2. Conjunto de Treino

Usando dois scripts em python (estão no repositório), organizamos o conjunto de dados da seguinte forma:

- Seq2Seq: (train foram usados no treino, test foram usados na validação)
    - **`train.a`** e **`test.a`** - cada linha possui uma fala que foi dita antes de uma fala do Homer (sim, estamos ignorando os casos em que a fala que foi dita antes não tenha sido falada para o Homer)
    -  **`train.b`** e **`test.b`** - cada linha possui uma fala que foi dita pelo Homer.
- Doc2Vec: (mesma coisa que os do Seq2Seq, mas todas as falas do Homer ficam antes)
    - **`questions.txt`** - mesmo caso dos arquivos **`.a`** do Seq2Seq
    - **`answers.txt`** - mesmo caso dos arquivos **`.b`** do Seq2Seq


In [0]:
# Fazendo o Drive estar disponível como pastas acessíveis para o python e shell
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
%%bash
# Copiando os dados de treino e teste do Drive para a pasta de trabalho - Seq2Seq
cp /content/gdrive/My\ Drive/homer_chatbot/homer_train.a ./train.a
cp /content/gdrive/My\ Drive/homer_chatbot/homer_train.b ./train.b
cp /content/gdrive/My\ Drive/homer_chatbot/homer_test.a ./test.a
cp /content/gdrive/My\ Drive/homer_chatbot/homer_test.b ./test.b

In [0]:
%%bash
# Copiando os dados de treino e teste do Drive para a pasta de trabalho - Doc2Vec
cp /content/gdrive/My\ Drive/homer_chatbot/questions.txt\
   /content/questions.txt
cp /content/gdrive/My\ Drive/homer_chatbot/answers.txt\
   /content/answers.txt

## 3. Modelo Seq2Seq

Tendo os arquivos organizados em seus devidos formatos, vamos falar sobre o primeiro modelo - [Seq2Seq](https://google.github.io/seq2seq/). Seq2Seq é um modelo criado pela Google para "Tradução Automática, Sumarização de Texto, **Modelagem Conversacional**, Legendas automáticas de Imagens, e mais". Ele é um modelo generativo, ou seja, gera dados novos a partir do aprendizado que ele teve com os dados de entrada.

Na verdade, vamos utilizar um modelo já construido usando Seq2Seq, o TernsorFlow-nmt, feito para tradução neural, mas ao invés de traduzir frases de uma língua para a outra, vamos traduzir frases para o Homer para frases do Homer.

In [0]:
%%bash
# Baixando o nmt (tf-seq2seq)
rm -rf /content/nmt_model
rm -rf nmt
git clone https://github.com/tensorflow/nmt/

Cloning into 'nmt'...


Isso funciona porque tradução neural consiste em abstrair o significado de uma frase em uma língua e traduzir este significado em outra língua, mas o mesmo princípio pode ser usado para chatbots: a rede neural abstrai o significado de uma fala, e a partir dele, busca responder de acordo com este significado, na mesma língua.

Na verdade são duas redes neurais em uma - um "encoder" - que aprende o significado da frase de entrada, representado por um vetor, e um "decoder", que recebe esse vetor, e aprende a responder a ele:

![imagens mal feitas são mais legais](https://drive.google.com/uc?id=1oF9AxZ-DpweMJ0vO_5gyaYwqKCSG-szN)

Nós dividimos o conjunto de entrada entre _treino_ e _validação_ para evitar o chamado "over-fitting" - quando o modelo "aprende demais". A seguir uma imagem que explica intuitivamente como um modelo de predição pode aprender demais:

![sim, fiz isso no paint](https://drive.google.com/uc?id=1mrF7OQNLWSALasnYL6As77i1XjFjqUNC)

Antes de treinar a rede neural, precisamos montar um _vocabulário_, um conjunto de todas as palavras que a rede neural pode utilizar para entender e responder. Podemos fazer isso apenas dando o conjunto de todas as palavras vistas no conjunto de dados, mas fazer isso criaria um vocabulário potencialmente muito grande, e sem certas conexões úteis entre palavras similares.

Uma maneira de resolver isto é dividindo as palavras em sub-palavras, por exemplo "loved" seria uma composição de "lov" e "ed", e "loving" seria uma composição de "lov" e "ing". Isso faz com que o modelo possa generalizar melhor para palavras novas e ainda diminui o tamanho do vocabulário.

Existem vários métodos de fazer isso, e o que vamos usar é o Byte-Pair-Encoding (BPE). Para isso, usamos o repositório subword-nmt:

In [0]:
%%bash
# Clonando o repositório subword-nmt
rm -rf subword-nmt
git clone https://github.com/b0noI/subword-nmt.git
cd subword-nmt
git checkout dbe97c8f95f14d06b2e46b8053e2e2f9b9bf804e

cd /content/

# Criando o vocabulário de palavras únicas a partir dos dados de treino
subword-nmt/learn_joint_bpe_and_vocab.py --input ./train.a ./train.b -s 50000 -o code.bpe --write-vocabulary vocab.train.bpe.a vocab.train.bpe.b

# Removendo os tabs inúteis dos vocabulários
sed -i '/\t/d' ./vocab.train.bpe.a
sed -i '/\t/d' ./vocab.train.bpe.b

# Fazendo novos arquivos de vocabulário (sem as frequências, pra usar no tf-seq2seq)
cat vocab.train.bpe.a | cut -f1 --delimiter=' ' > revocab.train.bpe.a
cat vocab.train.bpe.b | cut -f1 --delimiter=' ' > revocab.train.bpe.b

Tendo criado os vocabulários (um para os `.a` e outro para os `.b`), aplicamos ele nos conjuntos de treino.

In [0]:
%%bash
# Aplicando os vocabulários com sub-palavras em todos os arquivos (treino e teste)
subword-nmt/apply_bpe.py -c code.bpe --vocabulary vocab.train.bpe.a --vocabulary-threshold 5 < ./train.a > train.bpe.a
subword-nmt/apply_bpe.py -c code.bpe --vocabulary vocab.train.bpe.b --vocabulary-threshold 5 < ./train.b > train.bpe.b
subword-nmt/apply_bpe.py -c code.bpe --vocabulary vocab.train.bpe.a --vocabulary-threshold 5 < ./test.a > test.bpe.a
subword-nmt/apply_bpe.py -c code.bpe --vocabulary vocab.train.bpe.b --vocabulary-threshold 5 < ./test.b > test.bpe.b

Esta próxima célula de código só deve ser rodada se quiser carregar do Drive um modelo já
treinado anteriormente, para continuar o treinamento ou testar o modelo já treinado

In [0]:
# Carregando modelo treinado anteriormente
!rm -rf /content/nmt/nmt_model
!cp -r /content/gdrive/My\ Drive/homer_chatbot/nmt_model /content/nmt/nmt_model
!echo -e "\nloaded\n"

E agora, finalmente, o treinamento!

In [0]:
# Treinando o modelo seq2seq
import json

# Loop para treinar por 10.000 passos de cada vez,
# salvando no drive entre iterações.
for i in range(1,50):
  
  # Atualizando Parâmetro de Passos
  filename = '/content/nmt/nmt_model/hparams'
  
  with open(filename, 'r') as f:
    data = json.load(f)
    data["num_train_steps"] = i*20*500 + 457690

  get_ipython().system("rm " + filename)
  with open(filename, 'w') as f:
    json.dump(data, f, indent=4)
    
  # Treinando de fato
  !cd nmt && python3 -m nmt.nmt \
    --src=a --tgt=b \
    --vocab_prefix=../revocab.train.bpe \
    --train_prefix=../train.bpe \
    --dev_prefix=../test.bpe \
    --test_prefix=../test.bpe \
    --out_dir=nmt_model \
    --num_layers=2 \
    --num_gpus=1
  
  # Salvando o progesso
  !rm -rf /content/gdrive/My\ Drive/homer_chatbot/nmt_model
  !cp -r /content/nmt/nmt_model /content/gdrive/My\ Drive/homer_chatbot
  !echo -e "\nsaved\n"


loaded


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

# Job id 0
2019-06-01 17:49:46.028738: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-06-01 17:49:46.028932: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3444680 executing computations on platform Host. Devices:
2019-06-01 17:49:46.028961: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-01 17:49:46.187937: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-01 17:49:46.188455: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x3443fa0 executing computations on p

Após o treinamento, podemos criar uma função para conversar com a rede neural, fazendo inferências novas a partir de um arquivo `input.txt`:

In [0]:
def chatbot_seq2seq():
  quit = False
  while(quit == False):
    text = input("Me: ")
    
    if(text == "quit()"):
      quit = True
      
    else:
      with open("/content/input.txt", "w") as input_file:
        input_file.write(text)
  
      !/content/subword-nmt/apply_bpe.py -c /content/code.bpe --vocabulary /content/vocab.train.bpe.a --vocabulary-threshold 5 < /content/input.txt > /content/input.bpe
      !cd /content/nmt && python -m nmt.nmt \
        --out_dir=nmt_model \
        --inference_input_file=/content/input.bpe \
        --inference_output_file=/content/output.txt > /dev/null 2>&1
  
      with open("/content/output.txt", "r") as output:
        print('Homer: ', output.read().replace('@@ ', '').strip("@@\n"))
        print('\n')

In [0]:
# Célula para rodar a função de chatbot
chatbot_seq2seq()

Me: hey old friend
chatbot:  marge, i do that stupid du


Me: quit()


## 4. Modelo Doc2Vec

O modelo Doc2Vec é semelhante ao Seq2Seq no sentido de percepção de "sentido" para frases, mas ele só vai até aí. Dentro de si, ele utiliza o modelo Word2Vec, que atribui significado a palavras em vetores numéricos, mas vai um passo além e usa os significados de múltiplas palavras para inferir o significado de uma frase - um "**doc**umento".

![alt text](https://drive.google.com/uc?id=1jZ_Om2unMFtlodKWECXAb7w4OJY4TJLH)

Tendo o vetor que representa a frase de entrada, procuramos dentre todas as falas de Os Simpsons que foram faladas para o Homer, qual é a mais parecida, ou seja, qual vetor é mais similar (ou próximo) ao vetor gerado pelo modelo, e retornamos a resposta que o Homer deu para esta frase. Isso configura um modelo de diálogo "retrieval-based", ou seja, baseado em busca de frases já prontas, diferentemente dos modelos generativos, que geram texto novo.

A biblioteca que vamos usar para isso é a [gensim](https://radimrehurek.com/gensim/tutorial.html), também originalmente publicada pela Google, que serve para implementação de modelos Word2Vec e Doc2Vec.

Nessa versão, como não vamos gerar conteúdo novo, vamos criar um vocabulário com palavras inteiras mesmo, onde palavras que aparecem apenas uma vez são ignoradas e tratadas como palavras desconhecidas.

Rode a próxima célula de código se e somente se quiser carregar o modelo pré-treinado do Drive:

In [0]:
# Carregando modelo treinado anteriormente
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import multiprocessing
import os

!rm -rf /content/doc2vec.model
!rm -rf /content/doc2vec.model.docvecs.vectors_docs.npy
!cp /content/gdrive/My\ Drive/homer_chatbot/doc2vec.model /content/doc2vec.model
!cp /content/gdrive/My\ Drive/homer_chatbot/doc2vec.model.docvecs.vectors_docs.npy /content/doc2vec.model.docvecs.vectors_docs.npy

doc2vec_model = Doc2Vec.load("/content/doc2vec.model")

!echo -e "\nloaded\n"

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



loaded



E caso queira treinar...

In [0]:
# Treinando o modelo
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import multiprocessing
import os

# Só tendo certeza de que será usado o compilador C (para treinar mais rápido)
assert gensim.models.doc2vec.FAST_VERSION > -1

print('Fazendo uns paranauês matemáticos...')

# Precisamos do número de palavras no conjunto de treino
with open("questions.txt", "r") as f:
  n_words = len(f.read().split())

cores = multiprocessing.cpu_count()

# Criando o modelo com 200 dimensões de vetores, palavras que
# aparecem no mínimo 2 vezes e com o número certinho de cores
doc2vec_model = Doc2Vec(vector_size=200, min_count=2, workers=cores)
doc2vec_model.build_vocab(corpus_file="questions.txt")
doc2vec_model.train(corpus_file="questions.txt",
                    total_words=n_words,
                    epochs=100)

if not os.path.exists("models"):
    os.makedirs("models")

doc2vec_model.save('models/doc2vec.model')

# Salvando o modelo
!rm -rf /content/gdrive/My\ Drive/homer_chatbot/doc2vec.model
!cp /content/models/doc2vec.model /content/gdrive/My\ Drive/homer_chatbot/doc2vec.model
!cp /content/models/doc2vec.model.docvecs.vectors_docs.npy /content/gdrive/My\ Drive/homer_chatbot/doc2vec.model.docvecs.vectors_docs.npy
!echo -e "\nsaved\n"

print('Feito!')

Fazendo uns paranauês matemáticos...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



saved

Feito!


Para testar e ver o funcionamento do Doc2Vec, vamos fazer um "teste de sanidade" - retorno das 2 palavras mais parecidas com outra qualquer

In [0]:
# 'Sanity Test'
import warnings

# Um filtro de warnings chatos, pra não atrapalharem a apresentação...
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    # vetor com palavras mais parecidas
    print(doc2vec_model.wv.most_similar(['nice'])[0:2])

[('good', 0.46582120656967163), ('well,', 0.4623730182647705)]


... E a função de chatbot é feita usando o método `most_similar` no vetor inferido a partir de uma frase nova:

In [0]:
import warnings

with open("answers.txt") as f:
    answers = f.read().split("\n")

# A função de ChatBot em si
def chatbot_doc2vec():
    quit=False
    while quit == False:
        text = input('Me: ').lower()
        # Um comando de quit opcional
        if text == 'quit()':
            quit=True
        else:
            tokens = text.split()
            # Infere vetor para o texto que o modelo pode não ter visto ainda
            new_vector = doc2vec_model.infer_vector(tokens)
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                
                # 15248 é o último índice das linhas com respostas do Homer,
                # é até onde o modelo pode pegar frases para responder.
                index = doc2vec_model.docvecs.most_similar([new_vector], topn=1, clip_end=15248)

            # index é uma lista de tuplas (index no arquivo de treino, similaridade)
            print("Homer: ", answers[index[0][0]])
            print('\n')

In [0]:
chatbot_doc2vec()

Me: what's your favourite hobby?
chatbot:  oh... it's a special time in a boy's life when... gotta go! so come to the legless frog... if you want to get sick and die, and leave a big, garlicky corpse. p.s. parking was ample.


Me: when were you born?
chatbot:  no! you're wrong! check again!




KeyboardInterrupt: ignored

## 5. Experimentação

Com o chatbot em mãos, testamos as capacidades do nosso chatbot com outras pessoas, para validar o quão bom ele é. Fizemos o seguinte: apresentamos as duas versões do chatbot, explicando a diferença entre elas, mas sem dizer qual personagem é, e pedimos para a pessoa tentar descobrir qual é. Afinal, se o único propósito deste robô é ser o Homer Simpson, vamos testar o quão Homer Simpson ele é.

---

![alt text](https://drive.google.com/uc?id=12caEi1Yy5dMmT8qnJ9vL4DCgzUeS1VDN)

---

Testando com uma colega com quem eu nunca falei, ele me fala uma coisa dessas na primeira resposta - mas tudo bem, ela entendeu que ele não faz muito sentido. Conversando um pouco com a primeira versão, ela viu que seria melhor tentar com a outra, que podia fazer mais sentido.

Foi nesse momento que um outro colega que estava asstindo teve a ideia de perguntar qual o nome dele, e ele respondeu de acordo:

---

![alt text](https://drive.google.com/uc?id=1YJTqAXks42HHn5XoAqoXZomGoY5bhWuZ)

---

(note a desconsideração da máquina para com sua nova amiga no final)

---

![alt text](https://drive.google.com/uc?id=1yuqustjsMenJz9Q2amHHgt03-yp4I3VL)

---

Literalmente na primeira resposta, o Homer se entrega, gritando 'Bart!' - e isso que o colega estava super interessado em conversar com o chatbot...

---

![alt_text](https://drive.google.com/uc?id=130I4VgSE-4EMBDXMzKUMgW8-bqwnT7I8)

---

Ele parece gostar de chamar a pessoa com quem está interagindo de "Marge", o que faz sentido, pela alta porcentagem das falas do Homer que foram com a Marge.

---

![alt_text](https://drive.google.com/uc?id=1tbCUwDBshzN0tXrfc4LEWIcZf-N4p2VS)

---

Ele também parece não entender abreviações direito, respondendo melhor às frases mais gramaticamente corretas.

Mais alguns momentos relevantes:

---

![alt text](https://drive.google.com/uc?id=1dEY5Y49uEcjCaHDlWuveoG9uhfMkZH-a)

---

![alt text](https://drive.google.com/uc?id=1n5Piww_s4WS60NM3GpPcYezSCoE9pvN_)

---

Todos os diálogos completos da parte de experimentação estão no repositório do github.

## 6. Conclusão

Com os resultados no mínimo estranhos, concluímos que o conjunto de dados era pequeno demais para uma aprendizagem de contexto das palavras e suas relações boa o bastante, para ambos modelos. Mas também que existem modelos de redes neurais que são mais adequados para diferentes conjuntos de dados e objetivos de processamento de linguagem natural.

Na experimentação, vimos que os chatbots, apesar de tudo, têm uma semelhança com o Homer Simpson, com uma personalidade parecida e de vez em quando mencionando outros personagens da série. Isso é óbvio no caso do modelo Doc2Vec, mas ficamos surpresos com estes resultados do modelo Seq2Seq.

Para fazer uma ligação com o que aprendemos na cadeira, vamos entender como funcionam modelos de aprendizado de máquina, comparando com expressões regulares e autômatos:

Autômatos são criados a partir de algoritmos, que, assim como expressões regulares, podem ser criados de maneira racional para prever contextos diferentes de um diálogo. Porém isto está limitado à imaginação e observação do programador, de identificar e prever programáticamente tais contextos.

Porém algoritmos de aprendizado de máquina são diferentes, pois **aprendem** quais os melhores parâmetros para um determinado modelo, como as redes neurais. Neles, o trabalho do programador ~~(além de passar noites tentando entender como implementar)~~ é apenas escolher os parâmetros certos, que potencialmente serão relevantes para o resultado final, e deixar o modelo treinar pela quantidade de tempo certa.

## 7. Links Úteis

[Nosso repositório](https://github.com/mswlandi/homer-chatbot)

### Seq2Seq

[Nossa referência para o modelo Seq2Seq](https://blog.kovalevskyi.com/how-to-create-a-chatbot-with-tf-seq2seq-for-free-e876ea99063c)

[Página da Google do seq2seq](https://google.github.io/seq2seq/)

### Doc2Vec

[Nossa referência para o modelo Doc2Vec](https://towardsdatascience.com/how-to-build-an-easy-quick-and-essentially-useless-chatbot-using-your-own-text-messages-f2cb8b84c11d)

[A gentle Introduction to Doc2Vec](https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e)

[(Outro caderno Jupyter) - Tutorial de Doc2Vec](https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb)